In [2]:
%load_ext cython

In [6]:
import time

In [40]:
def timeit(func, *args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    return (time.time()-start)
時間=timeit

In [5]:
def mandelbrot_nocython(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break


In [3]:
%%cython -a
import numpy as np

def mandelbrot_cython(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [13]:
%%cython -a
import numpy as np

def mandelbrot_cython_types(int[:,::1] m,
                      int size,
                      int iterations):
    cdef int i, j, n
    cdef complex z, c
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if z.real**2 + z.imag**2 <= 100:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [19]:
size = 30
s = (size, size)
m = np.zeros(s, dtype=np.int32)
start = time.time()
mandelbrot_cython(m, size, 100000)
print(time.time()-start)

12.002677202224731


In [12]:
start = time.time()
mandelbrot_nocython(m, size, 10000)
print(time.time()-start)

1.4040005207061768


In [18]:
start = time.time()
mandelbrot_cython_types(m, size, 100000)
print(time.time()-start)

0.7699980735778809


In [27]:
from math import sqrt

class  Planet(object):
  def __init__(self):  
    self.x = 1.0
    self.y = 0.0
    self.z = 0.0
    self.vx=0.0
    self.vy= 0.5
    self.vz = 0.0
    self.m =1.0

def single_step(planet,dt):

    distance =sqrt(planet.x**2 + planet.y**2 + planet.x**2)    
    Fx = -planet.x/distance**3
    Fy = -planet.y/distance**3
    Fz = -planet.z/distance**3


    planet.x +=dt*planet.vx
    planet.y += dt*planet.vy
    planet.z += dt*planet.vz

    planet.vx += dt * Fx/planet.m
    planet.vy += dt * Fy/planet.m
    planet.vz += dt * Fz/planet.m

def step_time(planet,time_span,n_steps):   
    dt = time_span / n_steps

    for j in range(n_steps):
        single_step(planet,dt)

def main(iters = 1000000):   
    miPlanet = Planet()
    step_time(miPlanet,5000,iters)
timeit(main, iters=4000000)

6.7550225257873535


In [32]:
%%cython -a

cdef extern from "math.h":
 double sqrt(double x)

cdef class  Planet_c(object):
  cdef public double x,y,z,vx,vy,vz,m
  def __init__(self):  
    self.x = 1.0
    self.y = 0.0
    self.z = 0.0
    self.vx=0.0
    self.vy= 0.5
    self.vz = 0.0
    self.m =1.0

cdef void single_step(Planet_c planet, double dt):
    cdef double distance,Fx,Fy,Fz
    distance =sqrt(planet.x**2 + planet.y**2 + planet.x**2)    
    
    Fx = -planet.x/distance**3
    Fy = -planet.y/distance**3
    Fz = -planet.z/distance**3

    planet.x +=dt*planet.vx
    planet.y += dt*planet.vy
    planet.z += dt*planet.vz

    planet.vx += dt * Fx/planet.m
    planet.vy += dt * Fy/planet.m
    planet.vz += dt * Fz/planet.m

cdef  void step_time_c(Planet_c planet, double time_span, int n_steps):   

    dt = time_span / n_steps
    cdef int j
    for j in range(n_steps):
        single_step(planet,dt)

def main_c(iters = 1000000):   
 miPlanet = Planet_c()
 step_time_c(miPlanet,5000,iters)


In [41]:
sp = 時間(main, iters=10000000)
sc = 時間(main_c,iters=10000000)
print(f"Python: {sp} Cython: {sc} Ratio: {sp/sc}")#https://ipython-books.github.io/55-accelerating-python-code-with-cython/

Python: 17.07099747657776 Cython: 0.9960150718688965 Ratio: 17.139296340713187
